<a href="https://colab.research.google.com/github/harirm85/Hari_ML_workshop/blob/main/EDA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy the Jason file and set security
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#Download data
!kaggle competitions download -c home-credit-default-risk

import zipfile

# Specify the path to the downloaded zip file
zip_path = "/content/home-credit-default-risk.zip"

# Specify the directory to extract the files to
extract_path = "/content/home-credit-default-risk"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
app_data = pd.read_csv('/content/home-credit-default-risk/application_train.csv')
prev_data = pd.read_csv('/content/home-credit-default-risk/previous_application.csv')

print(app_data.shape)
print(prev_data.shape)

In [ ]:
prev_data.info(verbose=True, show_counts=True)